In [1]:
import warnings
warnings.filterwarnings('ignore',category=FutureWarning)
warnings.filterwarnings('ignore',category=DeprecationWarning)
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate, KFold, RepeatedKFold, RandomizedSearchCV
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, mean_absolute_percentage_error
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures, MinMaxScaler
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.inspection import permutation_importance
import time
import xgboost as xgb

In [2]:
data = pd.read_excel("../Dataset.xlsx", sheet_name=['Total Consumers'])
df = data['Total Consumers']
number_of_houses = len(df.columns)
df

,0,1,2,3,4,5,6,7,8,9,...,41,42,43,44,45,46,47,48,49,50
0,2.964,2.322959,1.544607,0.778310,1.962012,2.677445,0.237877,0.689194,0.358525,0.814643,...,0.898895,0.203825,0.221624,0.319531,0.830996,0.924987,0.219128,0.274880,0.990488,0.779475
1,2.584,2.371797,1.544607,0.778310,1.962012,2.733737,0.192929,0.558967,0.358525,0.660712,...,0.917793,0.165311,0.179747,0.319531,0.848467,0.944434,0.177722,0.222940,1.011313,0.795863
2,3.071,2.415961,1.319880,0.665072,1.676555,2.784640,0.382869,1.109272,0.377198,1.311186,...,0.934883,0.328060,0.356708,0.336174,0.864266,0.962019,0.352691,0.442426,1.030144,0.810682
3,2.694,2.302538,1.319880,0.665072,1.676555,2.653908,0.442052,1.280743,0.377198,1.513868,...,0.890992,0.378772,0.411848,0.336174,0.823691,0.916855,0.407209,0.510816,0.981781,0.772623
4,2.569,2.363063,0.913154,0.460128,1.159919,2.723669,0.192242,0.556976,0.668500,0.658358,...,0.914413,0.164722,0.179106,0.595793,0.845343,0.940956,0.177089,0.222146,1.007588,0.792932
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35131,1.664,2.244719,1.455982,0.733653,1.849437,2.587266,0.205654,0.595835,0.171793,0.704291,...,0.868619,0.176214,0.191602,0.153109,0.803007,0.893832,0.189444,0.237645,0.957128,0.753222
35132,1.659,2.136340,1.201186,0.605264,1.525786,2.462348,0.201219,0.582985,0.067223,0.689101,...,0.826680,0.172414,0.187470,0.059912,0.764237,0.850676,0.185359,0.232519,0.910916,0.716855
35133,1.664,2.192805,1.201186,0.605264,1.525786,2.527430,0.228585,0.662271,0.067223,0.782819,...,0.848530,0.195862,0.212966,0.059912,0.784436,0.873160,0.210568,0.264142,0.934992,0.735802
35134,1.697,1.446083,0.259545,0.130782,0.329682,1.666757,0.189302,0.548459,0.070958,0.648292,...,0.559578,0.162203,0.176368,0.063241,0.517310,0.575820,0.174381,0.218749,0.616596,0.485237


## Auxiliary functions

In [3]:
def plot_results(preds: np.array, actuals: np.array, title: str):
    
    plt.scatter(actuals, preds, c='b', label='predicted')
    plt.xlabel('actual')
    plt.ylabel('predicted')
    plt.title(title)
    plt.xlim(0, plt.xlim()[1])
    plt.ylim(0, plt.ylim()[1])
    _ = plt.plot([0, 100], [0, 100], '--r', label='y=x')
    plt.legend()
    plt.show()
    
    
def performance_metrics(preds: np.array, actuals: np.array):

    # calculate performance metrics
    
    mse = mean_squared_error(actuals, preds)
    mae = mean_absolute_error(actuals, preds)
    rmse = np.sqrt(mse)
    wape = np.sum(np.abs(preds - actuals)) / np.sum(np.abs(actuals)) * 100
    #mape = np.mean(np.abs((actuals - preds) / actuals)) * 100
    #mape = mae / actuals.mean()  
    r2 = r2_score(actuals, preds)

    # print performance metrics
    #print('MSE: %.4f' % mse)
    #print('RMSE: %.4f' % rmse)
    #print('MAE: %.4f' % mae)
    #print('WMAPE: %.4f' % wmape)
    print('WAPE: %.3f' % wape)
    #print('R2: %.4f' % r2)
    return mse, rmse, mae, wape, r2

def build_model(estimator, X_train: np.array, y_train: np.array, X_test: np.array):
    
    model = estimator
    model.fit(X_train, y_train.values.ravel())
    preds = model.predict(X_test)
    scores = cross_validate(estimator, X_train, y_train.values.ravel(), scoring=['r2', 'neg_root_mean_squared_error', 'neg_mean_squared_error', 'neg_mean_absolute_error'])
    return model, preds, scores

In [4]:
def total_averaged_metrics(metrics_list):
    
    print("Total Averaged MSE: {}".format(np.round(sum(i for i, j, k, l, m in metrics_list)/len(metrics_list),3)))
    print("Total Averaged RMSE: {}".format(np.round(sum(j for i, j, k, l, m in metrics_list)/len(metrics_list),3)))
    print("Total Averaged MAE: {}".format(np.round(sum(k for i, j, k, l, m in metrics_list)/len(metrics_list),3)))
    print("Total Averaged WAPE: {}".format(np.round(sum(l for i, j, k, l, m in metrics_list)*100/len(metrics_list),3)))
    print("Total Averaged R2: {}".format(np.round(sum(m for i, j, k, l, m in metrics_list)/len(metrics_list),3)))


def last_energy_points(df, number_timesteps):
    X = pd.DataFrame()
    for i in range(1, (number_timesteps + 1) ):
        X[f'Energy_{i*15}'] = df.shift(i)
    X.dropna(inplace=True)
    X.reset_index(drop=True, inplace=True)
    y = pd.DataFrame(df[number_timesteps:])
    y.reset_index(drop=True, inplace=True)
    y.columns = ["Energy"]
    return X, y

def build_predict_show(df, number_timesteps, estimator, train_size=0.8, start_timestep=1 ):
    full_start = time.time()
    metrics_list = []
    for i in range(start_timestep,(number_timesteps + 1)):
        start = time.time()
        print("\nNumber of features ", i)
        X, y = last_energy_points(df, i)

        X_train, X_test, y_train, y_test = train_test_split(X,y, train_size=train_size, random_state=42)

        model, preds, scores = build_model(estimator, X_train, y_train, X_test)
        mse, rmse, mae, wape, r2 = performance_metrics(preds, y_test.values.reshape(-1))
        metrics_list.append((mse,rmse,mae,wape,r2))
        print("CV MSE: {} ".format(np.round(scores['test_neg_mean_squared_error'].mean() * (-1),3)))
        print("CV RMSE: {} ".format(np.round(scores['test_neg_root_mean_squared_error'].mean() * (-1),3)))
        print("CV MAE: {} ".format(np.round(scores['test_neg_mean_absolute_error'].mean() * (-1),3)))
        print("CV R2: {} ".format(np.round(scores['test_r2'].mean(),3)))
        print("\nElapsed time: {} seconds".format(np.round(time.time() - start),3))
    print("\nFull Elapsed time: {} seconds".format(np.round(time.time() - full_start),3))
    return model, preds, scores, metrics_list

def show_graphic_per_timestep(metrics_list, number_timesteps, start_timestep=1):
    mse_list = []
    rmse_list = []
    mae_list = []
    wape_list = []
    r2_list = []

    for i in range(0,len(metrics_list)):
        mse_list.append(metrics_list[i][0])
        rmse_list.append(metrics_list[i][1])
        mae_list.append(metrics_list[i][2])
        wape_list.append(metrics_list[i][3] * 100)
        r2_list.append(metrics_list[i][4])
        
    plt.plot(range(start_timestep,(number_timesteps+1)), mse_list)
    plt.title('MSE per past timestep')
    plt.xlabel('Number of past timesteps')
    plt.ylabel('MSE')
    plt.show()
    
    plt.plot(range(start_timestep,(number_timesteps+1)), rmse_list)
    plt.title('RMSE per past timestep')
    plt.xlabel('Number of past timesteps')
    plt.ylabel('RMSE')
    plt.show()
    
    plt.plot(range(start_timestep,(number_timesteps+1)), mae_list)
    plt.title('MAE per past timestep')
    plt.xlabel('Number of past timesteps')
    plt.ylabel('MAE')
    plt.show()
    
    plt.plot(range(start_timestep,(number_timesteps+1)), wape_list)
    plt.title('WAPE per past timestep')
    plt.xlabel('Number of past timesteps')
    plt.ylabel('WAPE')
    plt.show()
    
    plt.plot(range(start_timestep,(number_timesteps+1)), r2_list)
    plt.title('R2 per past timestep')
    plt.xlabel('Number of past timesteps')
    plt.ylabel('R2')
    plt.show()
    

### Normalize data

In [5]:
values = df.values
scaler = MinMaxScaler()
df_scaled = scaler.fit_transform(values)
df_scaled = pd.DataFrame(df_scaled)
df_scaled

,0,1,2,3,4,5,6,7,8,9,...,41,42,43,44,45,46,47,48,49,50
0,0.060175,0.283908,0.248916,0.248916,0.248916,0.283908,0.077151,0.077151,0.119850,0.077151,...,0.283908,0.077151,0.077151,0.119850,0.283908,0.283908,0.077151,0.077151,0.283908,0.283908
1,0.052461,0.290083,0.248916,0.248916,0.248916,0.290083,0.062573,0.062573,0.119850,0.062573,...,0.290083,0.062573,0.062573,0.119850,0.290083,0.290083,0.062573,0.062573,0.290083,0.290083
2,0.062348,0.295667,0.212701,0.212701,0.212701,0.295667,0.124176,0.124176,0.126092,0.124176,...,0.295667,0.124176,0.124176,0.126092,0.295667,0.295667,0.124176,0.124176,0.295667,0.295667
3,0.054694,0.281326,0.212701,0.212701,0.212701,0.281326,0.143372,0.143372,0.126092,0.143372,...,0.281326,0.143372,0.143372,0.126092,0.281326,0.281326,0.143372,0.143372,0.281326,0.281326
4,0.052156,0.288978,0.147156,0.147156,0.147156,0.288978,0.062350,0.062350,0.223471,0.062350,...,0.288978,0.062350,0.062350,0.223471,0.288978,0.288978,0.062350,0.062350,0.288978,0.288978
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35131,0.033783,0.274015,0.234634,0.234634,0.234634,0.274015,0.066700,0.066700,0.057428,0.066700,...,0.274015,0.066700,0.066700,0.057428,0.274015,0.274015,0.066700,0.066700,0.274015,0.274015
35132,0.033681,0.260312,0.193573,0.193573,0.193573,0.260312,0.065262,0.065262,0.022472,0.065262,...,0.260312,0.065262,0.065262,0.022472,0.260312,0.260312,0.065262,0.065262,0.260312,0.260312
35133,0.033783,0.267452,0.193573,0.193573,0.193573,0.267452,0.074137,0.074137,0.022472,0.074137,...,0.267452,0.074137,0.074137,0.022472,0.267452,0.267452,0.074137,0.074137,0.267452,0.267452
35134,0.034453,0.173039,0.041826,0.041826,0.041826,0.173039,0.061397,0.061397,0.023720,0.061397,...,0.173039,0.061397,0.061397,0.023720,0.173039,0.173039,0.061397,0.061397,0.173039,0.173039


## Preprocessing

In [6]:
X,y = last_energy_points(df[0], 2)
print(X.shape, y.shape)
X

(35134, 2) (35134, 1)


,Energy_15,Energy_30
0,2.584,2.964
1,3.071,2.584
2,2.694,3.071
3,2.569,2.694
4,3.174,2.569
...,...,...
35129,1.670,1.686
35130,1.664,1.670
35131,1.659,1.664
35132,1.664,1.659


## Test model

### Gradient Boosting

In [7]:
start_timestep = 10
number_timesteps = 10

In [8]:
model_gb, preds_gb, scores_gb, metrics_list_gb = [],[],[],[]
start_t = time.time()
for house in range(0,number_of_houses):
    mo_gb, p_gb, s_gb, ml_gb = build_predict_show(df[house], number_timesteps, GradientBoostingRegressor(random_state=42), start_timestep=start_timestep)
    model_gb.append(mo_gb)
    preds_gb.append(p_gb)
    scores_gb.append(s_gb)
    metrics_list_gb.append(ml_gb)
print("\nFull Elapsed time: {} seconds".format(np.round(time.time() - start_t),3))


Number of features  10
WMAPE: 7.3731
CV MSE: 0.239 
CV RMSE: 0.477 
CV MAE: 0.269 
CV R2: 0.982 

Elapsed time: 39.0 seconds

Full Elapsed time: 39.0 seconds

Number of features  10
WMAPE: 27.3712
CV MSE: 0.259 
CV RMSE: 0.509 
CV MAE: 0.292 
CV R2: 0.747 

Elapsed time: 43.0 seconds

Full Elapsed time: 43.0 seconds

Number of features  10
WMAPE: 24.0933
CV MSE: 0.155 
CV RMSE: 0.394 
CV MAE: 0.165 
CV R2: 0.852 

Elapsed time: 27.0 seconds

Full Elapsed time: 27.0 seconds

Number of features  10
WMAPE: 24.0933
CV MSE: 0.039 
CV RMSE: 0.198 
CV MAE: 0.083 
CV R2: 0.852 

Elapsed time: 26.0 seconds

Full Elapsed time: 26.0 seconds

Number of features  10
WMAPE: 24.0932
CV MSE: 0.251 
CV RMSE: 0.5 
CV MAE: 0.209 
CV R2: 0.852 

Elapsed time: 26.0 seconds

Full Elapsed time: 26.0 seconds

Number of features  10
WMAPE: 27.3749
CV MSE: 0.345 
CV RMSE: 0.587 
CV MAE: 0.336 
CV R2: 0.747 

Elapsed time: 45.0 seconds

Full Elapsed time: 45.0 seconds

Number of features  10
WMAPE: 34.9401
CV M

KeyError: 51

In [ ]:
show_graphic_per_timestep(metrics_list_gb, number_timesteps, start_timestep)

#### Normalized

In [ ]:
model_gb_norm, preds_gb_norm, scores_gb_norm, metrics_list_gb_norm = [],[],[],[]
start_t = time.time()
for house in range(0,(number_of_houses + 1)):
    mo_gb, p_gb, s_gb, ml_gb = build_predict_show(df_scaled[house], number_timesteps, GradientBoostingRegressor(random_state=42), start_timestep=start_timestep)
    model_gb_norm.append(mo_gb)
    preds_gb_norm.append(p_gb)
    scores_gb_norm.append(s_gb)
    metrics_list_gb_norm.append(ml_gb)
print("\nFull Elapsed time: {} seconds".format(np.round(time.time() - start_t),3))

In [ ]:
show_graphic_per_timestep(metrics_list_gb_norm, number_timesteps, start_timestep)

### Random Forest

In [ ]:
model_rf, preds_rf, scores_rf, metrics_list_rf = [],[],[],[]
for house in range(0,(number_of_houses + 1)):
    mo_rf, p_rf, s_rf, ml_rf = build_predict_show(df[house], number_timesteps, RandomForestRegressor(random_state=42), start_timestep=start_timestep)
    model_rf.append(mo_rf)
    preds_rf.append(p_rf)
    scores_rf.append(s_rf)
    metrics_list_rf.append(ml_rf)

In [ ]:
show_graphic_per_timestep(metrics_list_rf, number_timesteps, start_timestep=start_timestep)

#### Normalized

In [ ]:
model_rf_norm, preds_rf_norm, scores_rf_norm, metrics_list_rf_norm = [],[],[],[]
start_t = time.time()
for house in range(0,(number_of_houses + 1)):
    mo_rf, p_rf, s_rf, ml_rf = build_predict_show(df_scaled[house], number_timesteps, RandomForestRegressor(random_state=42), start_timestep=start_timestep)
    model_rf_norm.append(mo_rf)
    preds_rf_norm.append(p_rf)
    scores_rf_norm.append(s_rf)
    metrics_list_rf_norm.append(ml_rf)
print("\nFull Elapsed time: {} seconds".format(np.round(time.time() - start_t),3))

In [ ]:
show_graphic_per_timestep(metrics_list_rf_norm, number_timesteps, start_timestep=start_timestep)

## XGBoost

In [ ]:
model_xgb, preds_xgb, scores_xgb, metrics_list_xgb = [],[],[],[]
start_t = time.time()
for house in range(0,(number_of_houses + 1)):
    print("\n House {}".format(house))
    mo_xgb, p_xgb, s_xgb, ml_xgb = build_predict_show(df[house], number_timesteps, xgb.XGBRegressor(max_depth=5, learning_rate=0.1, n_estimators=100, seed=42), start_timestep=start_timestep)
    model_xgb.append(mo_xgb)
    preds_xgb.append(p_xgb)
    scores_xgb.append(s_xgb)
    metrics_list_xgb.append(ml_xgb)
print("\XGBoost Elapsed time: {} seconds".format(np.round(time.time() - start_t),3))

In [ ]:
show_graphic_per_timestep(metrics_list_xgb, number_timesteps, start_timestep=start_timestep)

#### Normalized

In [ ]:
model_xgb_norm, preds_xgb_norm, scores_xgb_norm, metrics_list_xgb_norm = [],[],[],[]
start_t = time.time()
for house in range(0,(number_of_houses + 1)):
    mo_xgb, p_xgb, s_rf, ml_xgb = build_predict_show(df_scaled[house], number_timesteps, xgb.XGBRegressor(max_depth=5, learning_rate=0.1, n_estimators=100, seed=42), start_timestep=start_timestep)
    model_xgb_norm.append(mo_xgb)
    preds_xgb_norm.append(p_xgb)
    scores_xgb_norm.append(s_xgb)
    metrics_list_xgb_norm.append(ml_xgb)
print("\nFull Elapsed time: {} seconds".format(np.round(time.time() - start_t),3))

In [ ]:
show_graphic_per_timestep(metrics_list_xgb_norm, number_timesteps, start_timestep=start_timestep)

In [ ]:
print(min([sublist[3] for sublist in metrics_list_gb]))
print(min([sublist[3] for sublist in metrics_list_gb_norm]))
print(min([sublist[3] for sublist in metrics_list_rf]))
print(min([sublist[3] for sublist in metrics_list_rf_norm]))
print(min([sublist[3] for sublist in metrics_list_xgb]))
print(min([sublist[3] for sublist in metrics_list_xgb_norm]))

In [ ]:
def switch_metric(metric: str):
    if metric.lower() == "mse":
        m = 0
    elif metric.lower() == "rmse":
        m = 1
    elif metric.lower() == "mae":
        m = 2
    elif metric.lower() == "wape":
        m = 3
    elif metric.lower() == "r2":
        m = 4
    return m

In [ ]:
def find_best_model(metric: str, metrics_list):
    met = switch_metric(metric)
    min_value = min(map(lambda x: x[met], metrics_list))
    idx = [index for index, item in enumerate(map(lambda x: x[met], metrics_list)) if item == min_value]
    return metrics_list[idx[0]]

In [ ]:
all_metrics_lists = [metrics_list_gb, metrics_list_gb_norm, metrics_list_rf, metrics_list_rf_norm, metrics_list_xgb, metrics_list_xgb_norm]

In [ ]:
best_model = []
for model in all_metrics_lists:
    best_model.append(find_best_model("wape", model))

In [ ]:
X_names = ("WAPE", "R2")
X_axis = np.arange(len(X_names))
plt.bar(X_axis - 0.2, (best_model[0][3], best_model[0][4]), 0.2, label = 'Unnormalized GB')
plt.bar(X_axis, (best_model[2][3], best_model[2][4]), 0.2, label = 'Unnormalized RF')
plt.bar(X_axis + 0.2, (best_model[4][3], best_model[4][4]), 0.2, label = 'Unnormalized XGB')
plt.ylim(0,1)
plt.xticks(X_axis, X_names)
plt.xlabel("Metrics")
plt.ylabel("Vales")
plt.title("Unnormalized models")
plt.legend()
plt.show()

In [ ]:
X_names = ("WAPE", "R2")
X_axis = np.arange(len(X_names))
plt.bar(X_axis - 0.4, (best_model[1][3], best_model[1][4]), 0.2, label = 'Normalized GB')
plt.bar(X_axis + 0, (best_model[3][3], best_model[3][4]), 0.2, label = 'Normalized RF')
plt.bar(X_axis + 0.4, (best_model[5][3], best_model[5][4]), 0.2, label = 'Normalized XGB')

plt.ylim(0,1)
plt.xticks(X_axis, X_names)
plt.xlabel("Metrics")
plt.ylabel("Values")
plt.title("Normalized Models)")
plt.legend()
plt.show()

In [ ]:
X_names = ("WAPE", "R2")
X_axis = np.arange(len(X_names))
plt.bar(X_axis - 0.2, (best_model[0][3], best_model[0][4]), 0.4, label = 'Unnormalized GB')
plt.bar(X_axis + 0.2, (best_model[1][3], best_model[1][4]), 0.4, label = 'Normalized GB')
plt.ylim(0,1)
plt.xticks(X_axis, X_names)
plt.xlabel("Metrics")
plt.ylabel("Values")
plt.title("Normalized vs Unnormalized GB")
plt.legend()
plt.show()

In [ ]:
X_names = ("WAPE", "R2")
X_axis = np.arange(len(X_names))
plt.bar(X_axis - 0.2, (best_model[2][3], best_model[2][4]), 0.4, label = 'Unnormalized RF')
plt.bar(X_axis + 0.2, (best_model[3][3], best_model[3][4]), 0.4, label = 'Normalized RF')
plt.ylim(0,1)
plt.xticks(X_axis, X_names)
plt.xlabel("Metrics")
plt.ylabel("Values")
plt.title("Normalized vs Unnormalized RF")
plt.legend()
plt.show()

In [ ]:
X_names = ("WAPE", "R2")
X_axis = np.arange(len(X_names))
plt.bar(X_axis - 0.2, (best_model[4][3], best_model[4][4]), 0.4, label = 'Unnormalized XGB')
plt.bar(X_axis + 0.2, (best_model[5][3], best_model[5][4]), 0.4, label = 'Normalized XGB')
plt.ylim(0,1)
plt.xticks(X_axis, X_names)
plt.xlabel("Metrics")
plt.ylabel("Values")
plt.title("Normalized vs Unnormalized")
plt.legend()
plt.show()

## Best model

In [ ]:
test_score = np.zeros((params['n_estimators'],), dtype=np.float64)
for i, y_pred in enumerate(model.staged_predict(X_test)):
    test_score[i] = mean_squared_error(y_test, y_pred)

fig = plt.figure(figsize=(6, 6))
plt.subplot(1, 1, 1)
plt.title("Deviance")
plt.plot(
    np.arange(params['n_estimators']) + 1,
    model.train_score_,
    "b-",
    label="Training Set Deviance",
)
plt.plot(
    np.arange(params['n_estimators']) + 1, test_score, "r-", label="Test Set Deviance"
)
plt.legend(loc="upper right")
plt.xlabel("Boosting Iterations")
plt.ylabel("Deviance")
fig.tight_layout()
plt.show()

In [ ]:
feature_importance = model.feature_importances_
sorted_idx = np.argsort(feature_importance)
pos = np.arange(sorted_idx.shape[0]) + 0.5
fig = plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.barh(pos, feature_importance[sorted_idx], align="center")
plt.yticks(pos,X_test.columns)
plt.title("Feature Importance (MDI)")

result = permutation_importance(
    model, X_test, y_test, n_repeats=10, random_state=42, n_jobs=2
)
sorted_idx = result.importances_mean.argsort()
plt.subplot(1, 2, 2)
plt.boxplot(
    result.importances[sorted_idx].T,
    vert=False,
    labels=X_test.columns,
)
plt.title("Permutation Importance (test set)")
fig.tight_layout()
plt.show()

## Hyperparameter tuning

In [ ]:
grid = dict()
grid['n_estimators'] = [50, 100, 500]
grid['learning_rate'] = [0.001, 0.01, 0.1, 0.3]
grid['subsample'] = [0.5, 0.7, 1.0]
grid['max_depth'] = [3, 7, 9]

model = GradientBoostingRegressor()
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
#grid_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1, cv=cv)
grid_result = grid_search.fit(X, y.values.ravel())

print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))